In [ ]:
import duckdb
import pandas as pd
import numpy as np
import json
import os
import sys
sys.path.append('../analysis/')

import suffix_classes
import importlib

importlib.reload(suffix_classes)


In [ ]:
prefix = "" # adjust to your location
output_dir = ""

In [ ]:
# Update these paths to match your local setup
sqlite_2023 = os.path.join(prefix, "results/matching_2023.db")
sqlite_2024 = os.path.join(prefix, "results/matching_2024.db")
matching_file = "../dataset/apps/matching.json"

In [ ]:
duckdb.install_extension("sqlite")
duckdb.load_extension("sqlite")
duckdb_conn_2023 = duckdb.connect(database= sqlite_2023)
duckdb_conn_2024 = duckdb.connect(database= sqlite_2024)

In [ ]:
matching_ids = {}
with open(matching_file) as f:
    matching_ids = json.load(f)

In [ ]:
num_ios_apps = len(duckdb_conn_2023.sql("SELECT * FROM apps where platform = 'ios'").fetchall())

In [ ]:
num_android_apps = len(duckdb_conn_2023.sql("SELECT * FROM apps where platform = 'android'").fetchall())

In [ ]:
apps = duckdb_conn_2023.sql("SELECT app_name, platform FROM apps").fetchall()
android = [app[0] for app in apps if app[1] == "android"]
ios = [app[0] for app in apps if app[1] == "ios"]
matching_results = {}
for k,v in matching_ids.items():
    if v in matching_results.values():
        #print(k)
        pass
    if k in ios and v in android and v not in matching_results.values():
        matching_results[k] = v


In [ ]:
num_apps = len(duckdb_conn_2023.sql("SELECT * FROM apps").fetchall())

In [ ]:
existing_suffixes = duckdb_conn_2023.sql("SELECT DISTINCT suffix FROM files").fetchall()

In [ ]:
len(existing_suffixes)

In [ ]:
def get_formated(number, total):
    return f"{number:,} ({(number / total * 100):.2f}%)"

In [ ]:
base_query_file_suffix = "SELECT apps.app_name, apps.platform, files.file_size, files.file_name, files.file_path FROM files JOIN apps ON files.app_id = apps.id where file_name like $1"
hidden_files_query = "SELECT apps.app_name, apps.platform, files.file_size, files.file_name, files.file_path  FROM files JOIN apps ON files.app_id = apps.id where file_name like '.%'"
hidden_directories_query = "SELECT apps.app_name, apps.platform, files.file_size, files.file_name, files.file_path FROM files JOIN apps ON files.app_id = apps.id where file_path like '%/.%' and file_name not like '.%'"

only_android = " AND platform = 'android'"
only_ios = " AND platform = 'ios'"


all_suffix_query = "SELECT DISTINCT files.suffix FROM files JOIN apps ON files.app_id = apps.id where file_size >= 0"

get_suffix_file_count = """SELECT suffix, COUNT(files.id) AS file_count
FROM files JOIN apps on files.app_id = apps.id
WHERE platform = $1
GROUP BY suffix;"""

get_suffix_file_size = """SELECT suffix, SUM(files.file_size) AS total_size
FROM files JOIN apps on files.app_id = apps.id
WHERE platform = $1
GROUP BY suffix;"""


get_suffix_app_count = """SELECT suffix, COUNT(DISTINCT files.app_id)
FROM files JOIN apps on files.app_id = apps.id
WHERE platform = $1
GROUP BY files.suffix;"""

get_suffix_per_app_category = """
SELECT COUNT(DISTINCT files.app_id)
FROM files JOIN apps on files.app_id = apps.id
WHERE platform = $1 AND ( 
"""



In [ ]:
def get_per_platform(sql_query, arguments = [], duckdb_conn = duckdb_conn_2023):
    """
    Executes the given SQL query for all platforms, Android only, and iOS only.

    Args:
        sql_query (str): The SQL query to execute.
        arguments (list): List of arguments for parameterized queries.
        duckdb_conn: DuckDB connection object.

    Returns:
        tuple: (both, android, ios) where each is a list of query results.
    """
    both = duckdb_conn.execute(sql_query, arguments).fetchall()
    android = duckdb_conn.execute(sql_query + only_android, arguments).fetchall()
    ios = duckdb_conn.execute(sql_query + only_ios, arguments).fetchall()
    return both, android, ios

def get_unique_apps(results):
    """
    Extracts unique app identifiers from query results.

    Args:
        results (list): Query results, each containing (app_name, platform, ...).

    Returns:
        set: Unique app identifiers as "app_name - platform".
    """
    apps = set()
    for result in results:
        apps.add(result[0] + " - " + result[1])
    return apps

def get_unique_per_argument(results, index):
    """
    Extracts unique values from a specific column in the query results.

    Args:
        results (list): Query results.
        index (int): Index of the column to extract unique values from.

    Returns:
        set: Unique values from the specified column.
    """
    unique = set()
    for result in results:
        unique.add(result[index])
    return unique

def get_query_stats(query, arguments = [], unique_index = None, duckdb_conn = duckdb_conn_2023):
    """
    Runs a query per platform and computes unique app and argument statistics.

    Args:
        query (str): SQL query to execute.
        arguments (list): Arguments for the query.
        unique_index (int, optional): Index for extracting unique values.
        duckdb_conn: DuckDB connection object.

    Returns:
        list: [both, android, ios, both_apps, android_apps, ios_apps, both_unique, android_unique, ios_unique]
    """
    both, android, ios = get_per_platform(query, arguments, duckdb_conn=duckdb_conn)
    both_apps = get_unique_apps(both)
    android_apps = get_unique_apps(android)
    ios_apps = get_unique_apps(ios)

    both_unique = android_unique = ios_unique = set()
    if unique_index is not None:
        both_unique = get_unique_per_argument(both, unique_index)
        android_unique = get_unique_per_argument(android, unique_index)
        ios_unique = get_unique_per_argument(ios, unique_index)

    return [both, android, ios, both_apps, android_apps, ios_apps, both_unique, android_unique, ios_unique]    

def get_hidden_files(duckdb_conn = duckdb_conn_2023):
    """
    Gets statistics for hidden files (files starting with a dot).

    Args:
        duckdb_conn: DuckDB connection object.

    Returns:
        list: Query statistics for hidden files.
    """
    return get_query_stats(hidden_files_query, unique_index=3, duckdb_conn=duckdb_conn)

def get_hidden_files_filtered(duckdb_conn = duckdb_conn_2023):
    """
    Gets statistics for hidden files, excluding common system and git files.

    Args:
        duckdb_conn: DuckDB connection object.

    Returns:
        list: Query statistics for filtered hidden files.
    """
    not_ds_store_git_add = " AND file_name not like '.DS_Store' AND file_name not like '.gitkeep' AND file_name not like '.gitignore' AND file_name not like '.placeholder'  AND file_size > 0" 
    stats = (get_query_stats(hidden_files_query + not_ds_store_git_add, unique_index=3, duckdb_conn=duckdb_conn))
    return stats

def get_unique_hidden_directories(results):
    """
    Extracts unique hidden directory names from file paths.

    Args:
        results (iterable): Iterable of file paths.

    Returns:
        set: Unique hidden directory names.
    """
    hidden_directory = set()
    for directory in results:
        splitted = directory.split("/")
        for i in range(1, len(splitted)):
            if splitted[i].startswith(".") and i != len(splitted) -1:
                hidden_directory.add(splitted[i])
    return hidden_directory

def get_hidden_directory_stats(duckdb_conn = duckdb_conn_2023):
    """
    Gets statistics for hidden directories (directories starting with a dot).

    Args:
        duckdb_conn: DuckDB connection object.

    Returns:
        list: Query statistics for hidden directories, including unique hidden directory names.
    """
    stats = get_query_stats(hidden_directories_query, unique_index=4, duckdb_conn=duckdb_conn)
    stats[6] = get_unique_hidden_directories(stats[6])
    stats[7] = get_unique_hidden_directories(stats[7])
    stats[8] = get_unique_hidden_directories(stats[8])
    return stats


In [ ]:
def count_result_to_map(results):
    """
    Converts a list of (key, value) tuples into a dictionary.

    Args:
        results (list): List of tuples, where each tuple is (key, value).

    Returns:
        dict: Dictionary mapping keys to values.
    """
    result_map = {}
    for result in results:
        if result[0] not in result_map:
            result_map[result[0]] = result[1]
    return result_map

def get_all_suffix_table(duckdb_conn = duckdb_conn_2023):
    """
    Builds a DataFrame summarizing file statistics per suffix for Android and iOS.

    Args:
        duckdb_conn: DuckDB connection object.

    Returns:
        pd.DataFrame: DataFrame indexed by statistic type, with columns as suffixes.
    """
    results_per_suffix = {}
    suffixes = duckdb_conn.execute(all_suffix_query).fetchall()
    size_android = count_result_to_map(duckdb_conn.execute(get_suffix_file_size, ["android"]).fetchall())
    size_ios = count_result_to_map(duckdb_conn.execute(get_suffix_file_size, ["ios"]).fetchall())
    files_android = count_result_to_map(duckdb_conn.execute(get_suffix_file_count, ["android"]).fetchall())
    files_ios = count_result_to_map(duckdb_conn.execute(get_suffix_file_count, ["ios"]).fetchall())
    apps_android = count_result_to_map(duckdb_conn.execute(get_suffix_app_count, ["android"]).fetchall())
    apps_ios = count_result_to_map(duckdb_conn.execute(get_suffix_app_count, ["ios"]).fetchall())

    for suffix in suffixes:
        suffix = suffix[0]
        results_per_suffix[suffix] = [
            files_android.get(suffix, 0),
            size_android.get(suffix, 0),
            apps_android.get(suffix, 0),
            files_ios.get(suffix, 0),
            size_ios.get(suffix, 0),
            apps_ios.get(suffix, 0)
        ]
    
    return pd.DataFrame(
        results_per_suffix,
        index=["Android Files", "Android Size", "Android Apps", "iOS Files", "iOS Size", "iOS Apps"]
    )


In [ ]:
def get_suffix_stat_table(duckdb_conn = duckdb_conn_2023):
    """
    Aggregates file statistics per suffix category and per individual suffix.

    Args:
        duckdb_conn: DuckDB connection object.

    Returns:
        tuple:
            - pd.DataFrame: Aggregated statistics per suffix category (index: stat type, columns: category).
            - pd.DataFrame: Statistics per individual suffix (index: stat type, columns: suffix).
    """
    # Get all suffix statistics as a dictionary
    all_suffix_results = get_all_suffix_table(duckdb_conn = duckdb_conn).to_dict()    

    results = {}
    results_per_suffix = {}
    suffixes = set()
    other = set()

    # Aggregate stats for each suffix category defined in suffix_classes.all_types
    for k, v in suffix_classes.all_types.items():
        android_files = 0 
        ios_files = 0
        android_size = 0
        ios_size = 0
        query = ""
        for value in v:
            try:
                # Sum up stats for each suffix in the category
                android_files += all_suffix_results[value]["Android Files"]
                android_size += all_suffix_results[value]["Android Size"]
                ios_files += all_suffix_results[value]["iOS Files"]
                ios_size += all_suffix_results[value]["iOS Size"]
                results_per_suffix[value] = [
                    all_suffix_results[value]["Android Files"],
                    all_suffix_results[value]["Android Size"],
                    all_suffix_results[value]["Android Apps"],
                    all_suffix_results[value]["iOS Files"],
                    all_suffix_results[value]["iOS Size"],
                    all_suffix_results[value]["iOS Apps"]
                ]
                query += "OR suffix = '" + value + "' "
                suffixes.add(value)
            except KeyError:
                # Skip suffixes not present in the results
                pass
        try:
            # Remove leading 'OR' and get app counts for the category
            query = query[2:]
            android_apps = duckdb_conn.execute(get_suffix_per_app_category + query + ");", ["android"]).fetchall()[0][0]
            ios_apps = duckdb_conn.execute(get_suffix_per_app_category + query + ");", ["ios"]).fetchall()[0][0]
            results[k] = [android_files, android_size, android_apps, ios_files, ios_size, ios_apps]
        except:
            # Skip if query fails
            pass

    # Identify suffixes not in any category ("other")
    for suffix in duckdb_conn.execute(all_suffix_query).fetchall():
        if suffix[0] not in suffixes:
            other.add(suffix[0])
    
    # Aggregate stats for "other" and ".split" suffixes
    android_files = 0
    android_size = 0
    ios_files = 0
    ios_size = 0
    query = ""
    split_ios = 0
    split_android = 0
    split_android_size = 0
    split_ios_size = 0
    split_query = ""
    for suffix in other:
        if "'" in suffix or '"' in suffix:
            continue
        if ".split" in suffix:
            split_query += "OR suffix = '" + suffix + "' "
            split_android += all_suffix_results[suffix]["Android Files"]
            split_ios += all_suffix_results[suffix]["iOS Files"]
            split_android_size += all_suffix_results[suffix]["Android Size"]
            split_ios_size += all_suffix_results[suffix]["iOS Size"]
        else:
            # Optionally print suffixes with many apps
            if all_suffix_results[suffix]["Android Apps"] > 40 or all_suffix_results[suffix]["iOS Apps"] > 40:
                print(suffix)
                print(all_suffix_results[suffix])
            android_files += all_suffix_results[suffix]["Android Files"]
            android_size += all_suffix_results[suffix]["Android Size"]
            ios_files += all_suffix_results[suffix]["iOS Files"]
            ios_size += all_suffix_results[suffix]["iOS Size"]
            query += "OR suffix = '" + suffix + "' "
    query = query[2:]
    split_query = split_query[2:]
    android_apps = duckdb_conn.execute(get_suffix_per_app_category + query + ");", ["android"]).fetchall()[0][0]
    ios_apps = duckdb_conn.execute(get_suffix_per_app_category + query + ");", ["ios"]).fetchall()[0][0]
    results["other"] = [android_files, android_size, android_apps, ios_files, ios_size, ios_apps]
    results["split"] = [
        split_android, split_android_size,
        duckdb_conn.execute(get_suffix_per_app_category + split_query + ");", ["android"]).fetchall()[0][0],
        split_ios, split_ios_size,
        duckdb_conn.execute(get_suffix_per_app_category + split_query + ");", ["ios"]).fetchall()[0][0]
    ]

    # Return two DataFrames: one for categories, one for individual suffixes
    return (
        pd.DataFrame(results, index=["Android Files", "Android Size", "Android Apps", "iOS Files", "iOS Size", "iOS Apps"]),
        pd.DataFrame(results_per_suffix, index=["Android Files", "Android Size", "Android Apps", "iOS Files", "iOS Size", "iOS Apps"])
    )


In [ ]:
# Hidden files and folders
def get_df_for_dot_results(results, name):
    # [both, android, ios, both_apps, android_apps, ios_apps, both_unique, android_unique, ios_unique]    
    tmp_result = {name: [len(results[1]), len(results[4]), len(results[2]), len(results[5])]}
    return pd.DataFrame(tmp_result, index=["Android Files", "Android Apps", "iOS Files", "iOS Apps"])




In [ ]:
def get_number_of_files(platform=None, duckdb_conn = duckdb_conn_2023):
    query = "SELECT COUNT(files.id) FROM files, apps WHERE files.app_id = apps.id"
    if platform == 'android':
        query += only_android
    elif platform == 'ios':
        query += only_ios
    return duckdb_conn.execute(query).fetchall()[0][0]


def get_size_of_files(platform=None, duckdb_conn = duckdb_conn_2023):
    query = "SELECT SUM(files.file_size) FROM files, apps WHERE files.app_id = apps.id"
    if platform == 'android':
        query += only_android
    elif platform == 'ios':
        query += only_ios
    return duckdb_conn.execute(query).fetchall()[0][0]

def get_number_of_apps(platform=None, duckdb_conn = duckdb_conn_2023):
    query = "SELECT COUNT(apps.id) FROM apps WHERE apps.id > 0"
    if platform == 'android':
        query += only_android
    elif platform == 'ios':
        query += only_ios
    return duckdb_conn.execute(query).fetchall()[0][0]

In [ ]:
def format_data_frame(data_frame_to_format, sort_index = True, android_apps = 0, ios_apps = 0):
    total_ios_files = get_number_of_files(platform="ios")
    total_android_files = get_number_of_files(platform="android")
    total_ios_app_size = get_size_of_files(platform="ios")
    total_android_app_size = get_size_of_files(platform="android")
    total_ios_apps = get_number_of_apps(platform="ios")
    total_android_apps = get_number_of_apps(platform="android")
    formated_df = data_frame_to_format.transpose()
    if sort_index:
        formated_df = formated_df.sort_index()


    for column in formated_df.columns:
        if 'Files' in column:
            total_files = total_android_files if 'Android' in column else total_ios_files
            formated_df[column] = formated_df[column].apply(lambda x: f"{x:,} ({(x / total_files * 100):.2f}%)")
        elif 'Apps' in column:
            if android_apps ==0:
                total_apps = 10332
                formated_df[column] = formated_df[column].apply(lambda x: f"{x:,} ({(x / total_apps * 100):.2f}%)")
            else:
                formated_df[column] = formated_df[column].apply(lambda x: f"{x:,} ({(x / android_apps if 'Android' in column else ios_apps  * 100):.2f}%)")
        elif 'Size' in column:
            total_size = total_android_app_size if 'Android' in column else total_ios_app_size
            formated_df[column] = formated_df[column].apply(lambda x: f"{x/1000000:,} ({(x / total_size * 100):.2f}%)")
    return formated_df


In [ ]:
df_suffix, df_suffix_detail = get_suffix_stat_table()

In [ ]:
df_all = df_suffix

In [ ]:
df_all

In [ ]:
format_data_frame(df_all)

In [ ]:
format_data_frame(df_suffix_detail, sort_index=False).to_csv(os.path.join(output_dir, "suffix_detail.csv"))

In [ ]:
format_data_frame(df_all, sort_index=False).to_csv(os.path.join(output_dir, "suffix.csv"))

In [ ]:
full_suffix_table = get_all_suffix_table()

In [ ]:
format_data_frame(full_suffix_table).to_csv(os.path.join(output_dir, "full_suffix_table.csv"))

In [ ]:
print(format_data_frame(df_all).to_latex())

In [ ]:
def fetch_all_from_queries(query, duckdb_conn = duckdb_conn_2023):
    return duckdb_conn.execute(query).fetchall()

In [ ]:
archive_public_suffix = fetch_all_from_queries("SELECT distinct(apps.id, apps.platform) from apps join files on apps.id = files.app_id where files.file_name = 'publicsuffixes.gz'")
text_multidexversion = fetch_all_from_queries("SELECT distinct(apps.id, apps.platform) from apps join files on apps.id = files.app_id where files.file_name = 'androidsupportmultidexversion.txt'")


In [ ]:
def get_stats(results):
    ios = set()
    android = set()
    for result in results:
        result = result[0]
        if result[1] == "android":
            android.add(result[0])
        else:
            ios.add(result[0])
    print(f"Android: {get_formated(len(android), num_android_apps)}")
    print(f"iOS: {get_formated(len(ios), num_ios_apps)}")

In [ ]:
get_stats(archive_public_suffix)
get_stats(text_multidexversion)

In [ ]:
# https://github.com/dyne/file-extension-list
# https://github.com/jddinneen/file-extension-categoriser
count = 0
for k,v in suffix_classes.all_types.items():
    count += len(v)

print(count)
print(len(suffix_classes.all_types))

In [ ]:
ios_exe = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.exe' and apps.platform = 'ios'")
print("ios exe", get_formated(len(ios_exe), num_ios_apps))
ios_dll = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.dll' and apps.platform = 'ios'")
print("ios dll", get_formated(len(ios_dll), num_ios_apps))
ios_dll_and_exe = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.dll' or file_name like '%.exe') and apps.platform = 'ios'")
print("ios dll and exe", get_formated(len(ios_dll_and_exe), num_ios_apps))

android_exe = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.exe' and apps.platform = 'android'")
android_dll = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.dll' and apps.platform = 'android'")
android_exe_dll = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.dll' or file_name like '%.exe') and apps.platform = 'android'")
print("android exe", get_formated(len(android_exe), num_android_apps))
print("android dll", get_formated(len(android_dll), num_android_apps))
print("android exe and dll", get_formated(len(android_exe_dll), num_android_apps))


both_exe = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.exe' ")
both_dll = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.dll' ")
both_exe_dll = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.dll' or file_name like '%.exe')")
print("both exe", get_formated(len(both_exe), num_apps))
print("both dll", get_formated(len(both_dll), num_apps))
print("both exe and dll", get_formated(len(both_exe_dll), num_apps))






In [ ]:
aspx = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.aspx' ")
print("aspx", get_formated(len(aspx), num_apps))
aspx_android = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.aspx' and apps.platform = 'android'")
print("aspx android", get_formated(len(aspx_android), num_android_apps))
aspx_ios = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where file_name like '%.aspx' and apps.platform = 'ios'")
print("aspx ios", get_formated(len(aspx_ios), num_ios_apps))

In [ ]:
exe_without_xarmin = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.exe') and apps.id NOT IN (SELECT DISTINCT apps.id FROM files join apps on files.app_id = apps.id where file_name = 'Xamarin.Essentials.dll' or file_name like '%Xamarin%' or file_name like '%xarmin%' )")
dll_without_xarmin = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.dll') and apps.id NOT IN (SELECT DISTINCT apps.id FROM files join apps on files.app_id = apps.id where file_name = 'Xamarin.Essentials.dll' or file_name like '%Xamarin%' or file_name like '%xarmin%'  )")


dll_and_exe_without_xarmin = fetch_all_from_queries("SELECT DISTINCT apps.id, apps.app_name, apps.platform FROM files join apps on files.app_id = apps.id where (file_name like '%.dll' or file_name like '%.exe') and apps.id NOT IN (SELECT DISTINCT apps.id FROM files join apps on files.app_id = apps.id where file_name = 'Xamarin.Essentials.dll' or file_name like '%Xamarin%' or file_name like '%xarmin%' )")



In [ ]:
print(get_formated(len(exe_without_xarmin), num_apps))
print(get_formated(len(dll_without_xarmin), num_apps))
print(get_formated(  len(dll_and_exe_without_xarmin), num_apps))

print(get_formated( len( both_exe_dll) - len(dll_and_exe_without_xarmin), num_apps))


In [ ]:
all_ios_on_android = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'android' AND (mime_type = 'application/x-mach-binary') ORDER BY apps.app_name;")


print("all ios on android", get_formated(len(all_ios_on_android), num_android_apps))

In [ ]:
all_ios_on_android = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'android' AND (mime_type = 'application/x-mach-binary') ORDER BY apps.app_name;", duckdb_conn=duckdb_conn_2024)


print("all ios on android", get_formated(len(all_ios_on_android), num_android_apps))

In [ ]:
jar_dex_apk_in_ios = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.jar' or files.suffix = '.apk' or files.suffix = '.dex') and apps.id not in (SELECT distinct files.app_id from files where files.file_name like '%robovm%' or files.file_name like '%adapter-mediation%' or files.file_name = 'application.jar') ORDER BY apps.app_name;")


total = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.jar' or files.suffix = '.apk' or files.suffix = '.dex') ORDER BY apps.app_name;")

jar_in_ios = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.jar') ORDER BY apps.app_name;")
dex_in_ios = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.dex') ORDER BY apps.app_name;")
apk_in_ios = fetch_all_from_queries("SELECT distinct apps.app_name FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.apk') ORDER BY apps.app_name;")
print("jar in ios", get_formated(len(jar_in_ios), num_ios_apps))
print("dex in ios", get_formated(len(dex_in_ios), num_ios_apps))
print("apk in ios", get_formated(len(apk_in_ios), num_ios_apps))
print("total", get_formated(len(total), num_ios_apps))



In [ ]:
number_robovm_and_gdx = fetch_all_from_queries("SELECT distinct files.app_id from files where files.file_name like '%robovm%' or files.file_name like '%adapter-mediation%' or files.file_name = 'application.jar'")

print("all compiled java without robovm and gdx in ios", get_formated(len(jar_dex_apk_in_ios), num_ios_apps))
print("robovm and gdx in ios", get_formated(len(number_robovm_and_gdx), num_ios_apps))
len(total) - len(jar_dex_apk_in_ios)


In [ ]:
review_remaining = "SELECT distinct apps.app_name, files.file_path FROM files JOIN apps ON files.app_id = apps.id WHERE apps.platform = 'ios' AND (files.suffix = '.jar' or files.suffix = '.apk' or files.suffix = '.dex') and files.app_id not in (SELECT distinct files.app_id from files where files.file_name like '%robovm%' or files.file_name like '%adapter-mediation%' or files.file_name = 'application.jar') ORDER BY apps.app_name;"

In [ ]:
limit = 10
code_suffix = [".kt", ".java", ".swift", ".c", ".cpp", ".go", ".m"]
script_suffix = [".sh", ".lua", ".py", ".ms", ".pyc", ".vs", ".ps1", ".bash"]

def only_platform(result, platform):
    """
    Filters results to include only entries for the specified platform.

    Args:
        result (list): List of tuples, where the second element is the platform.
        platform (str): Platform to filter by ('android' or 'ios').

    Returns:
        list: Filtered list containing only entries for the specified platform.
    """
    return [x for x in result if x[1] == platform]

def matching(result):
    """
    Finds apps present on both Android and iOS platforms.

    Args:
        result (list): List of tuples, where the first element is app_id and the second is platform.

    Returns:
        list: List of app_ids present on both platforms.
    """
    # matching_ids must be defined elsewhere in the notebook
    android = set()
    ios = set()
    for x in result:
        if x[1] == "android":
            android.add(x[0])
        else:
            ios.add(x[0])

    matching_apps = set()
    for app in ios:
        if matching_ids[app] in android:
            matching_apps.add(app)
    return list(matching_apps)

def get_code_stats(result_dict):
    """
    Prints statistics for code-related files per platform and overall.

    Args:
        result_dict (dict): Dictionary mapping suffix to list of (app_name, platform) tuples.
    """
    total = set()
    for key, value in result_dict.items():
        print(f"{key} Both: {get_formated(len(matching(value)), num_android_apps)}")
        print(f"{key} Android: {get_formated(len(only_platform(value, 'android')), num_android_apps)}")
        print(f"{key} iOS: {get_formated(len(only_platform(value, 'ios')), num_ios_apps)}")
        print(f"{key}: {get_formated(len(value) - len(matching(value)), num_apps)}")

        for v in value:
            total.add(v)
    print(f"{key} Both: {get_formated(len(matching(total)), num_android_apps)}")
    print(f"Total Android: {get_formated(len(only_platform(total, 'android')), num_android_apps)}")
    print(f"Total iOS: {get_formated(len(only_platform(total, 'ios')), num_ios_apps)}")
    print(f"Total: {get_formated(len(total) - len(matching(total)), num_apps)}")

def get_apps_with_code(code_suffix, limit = 10, limit_2 = 3, with_files = False, duckdb_conn = duckdb_conn_2023):
    """
    Retrieves apps containing code files with specific suffixes.

    Args:
        code_suffix (list): List of code file suffixes to search for.
        limit (int): Minimum number of files per app to include.
        limit_2 (int): Maximum number of duplicate file names allowed.
        with_files (bool): If True, include file paths in the result.
        duckdb_conn: DuckDB connection object.

    Returns:
        dict: Mapping from suffix to list of (app_name, platform[, file_path]) tuples.
    """
    result = {}
    select_part = "SELECT DISTINCT app_name, platform"
    if with_files:
        select_part = select_part + ", file_path"
    for code in code_suffix:
        base_query = (
            f"{select_part} FROM files join apps on files.app_id = apps.id "
            f"where suffix = '{code}' and app_id in ("
            f"SELECT app_id FROM files WHERE suffix = '{code}' "
            f"and file_path not like '%META-INF%' and file_path not like '%javax%' "
            f"and file_path not like '%androidx%' and mime_type like 'text%' "
            f"and file_name in (SELECT file_name FROM files WHERE suffix = '{code}' "
            f"and mime_type like 'text%' GROUP By file_name HAVING count(*) < {limit_2}) "
            f"GROUP By app_id HAVING count(*) > {limit});"
        )
        result[code] = duckdb_conn.sql(base_query).fetchall()
    return result

def get_apps_with_scripts(script_suffix, limit = 3, with_files = False, duckdb_conn = duckdb_conn_2023):
    """
    Retrieves apps containing script files with specific suffixes.

    Args:
        script_suffix (list): List of script file suffixes to search for.
        limit (int): Maximum number of duplicate file names allowed.
        with_files (bool): If True, include file paths in the result.
        duckdb_conn: DuckDB connection object.

    Returns:
        dict: Mapping from suffix to list of (app_name, platform[, file_path]) tuples.
    """
    result = {}
    select_part = "SELECT DISTINCT app_name, platform"
    if with_files:
        select_part = select_part + ", file_path"
    for code in script_suffix:
        base_query = (
            f"{select_part} FROM files join apps on files.app_id = apps.id "
            f"where suffix = '{code}' and file_name in ("
            f"SELECT file_name FROM files WHERE suffix = '{code}' "
            f"and mime_type like 'text%' GROUP By file_name HAVING count(*) < {limit});"
        )
        result[code] = duckdb_conn.sql(base_query).fetchall()
    return result


In [ ]:
code_stats = get_apps_with_code(code_suffix)
script_stats = get_apps_with_scripts(script_suffix)



In [ ]:
get_code_stats(code_stats)
get_code_stats(script_stats)


In [ ]:
get_apps_with_code(code_suffix, with_files=True)


In [ ]:
swiftpm = fetch_all_from_queries("SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where files.file_path like '%/.swiftpm/%';")
spm_versions = fetch_all_from_queries("SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where files.file_name = 'spm_versions.json';")
podfile = fetch_all_from_queries("SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where files.file_name like 'Podfile%';")
package_swift = fetch_all_from_queries("SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where files.file_name = 'Package.swift';")



In [ ]:
code_stats["swiftpm"] = swiftpm
code_stats["spm_versions"] = spm_versions
code_stats["podfile"] = podfile
code_stats["package_swift"] = package_swift

In [ ]:
get_code_stats({"podfile": podfile, ".swift": code_stats[".swift"]})

In [ ]:
get_code_stats({"swiftpm": swiftpm, ".swift": code_stats[".swift"]})

In [ ]:
get_code_stats({"swiftpm": swiftpm, "spm_versions": spm_versions, "package_swift": package_swift, ".swift": code_stats[".swift"]})

In [ ]:
def get_compiled_lua(limit = 3):
    query = f"SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where (suffix = '.lua' or suffix = '.luac') and file_name in (SELECT file_name  FROM files WHERE (suffix = '.lua' or suffix = '.luac') and mime_type like 'application%' GROUP By file_name HAVING count(*) < {limit});"
    result = fetch_all_from_queries(query)
    return {"lua-c": result}

luac = get_compiled_lua()

In [ ]:
get_code_stats(luac)

In [ ]:
java_kotlin_swift_files = "SELECT distinct apps.app_name, apps.platform, files.suffix, files.file_name FROM files JOIN apps ON files.app_id = apps.id WHERE  (files.suffix = '.php' or files.suffix = '.vue' or files.suffix = '.ts') ORDER BY apps.app_name;"

for item in fetch_all_from_queries(java_kotlin_swift_files):
    print(item)

In [ ]:
java_kotlin_swift_files = "SELECT distinct apps.app_name, apps.platform, files.suffix, files.file_name FROM files JOIN apps ON files.app_id = apps.id WHERE  (files.suffix = '.sh') ORDER BY apps.app_name;"

for item in fetch_all_from_queries(java_kotlin_swift_files):
    print(item)

In [ ]:
def get_full_stats(results, len_android = 0, len_ios = 0):
    android = set()
    ios = set()
    for result in results:
        if result[1] == "android":
            android.add(result[0])
        else:
            ios.add(result[0])
    if len_android == 0:
        len_android = num_android_apps
    if len_ios == 0:
        len_ios = num_ios_apps
    print(f"Android: {get_formated(len(android), num_android_apps)}")
    print(f"iOS: {get_formated(len(ios), num_ios_apps)}")
    print(f"Total: {get_formated(len(android) + len(ios), num_apps)}")

In [ ]:
import os
all_markdown = "SELECT distinct apps.app_name, apps.platform, files.suffix, files.file_name, files.file_path FROM files JOIN apps ON files.app_id = apps.id WHERE  (files.suffix = '.md' or files.suffix = '.mdown') ORDER BY apps.app_name;"

def get_markdown_stats(duckdb_conn = duckdb_conn_2023):

    all_markdown_in_node_modules = "SELECT distinct apps.app_name, apps.platform FROM files JOIN apps ON files.app_id = apps.id WHERE  (files.suffix = '.md' or files.suffix = '.mdown') and files.file_path like '%node_modules%' ORDER BY apps.app_name;"

    all_markdown_in_node_modules = fetch_all_from_queries(all_markdown_in_node_modules, duckdb_conn = duckdb_conn)

    third_party = set()
    potential_added_information = set()
    styling = set()


    for item in fetch_all_from_queries(all_markdown):
        if "cordova" in item[4].lower() or "node_modules" in item[4].lower() or "frameworks" in item[4].lower() or "www" in item[4].lower() or "meta-inf" in item[4].lower():
            third_party.add(( item[0], item[1]) ) # , item[3],item[4]
            continue

        dirname = os.path.dirname(item[4]).split("/")[-1]
        if ".app" in dirname or dirname == "android" or dirname.endswith(".android") or dirname.endswith("_android"):
            # Top level directory
            potential_added_information.add((item[0], item[1])) # , item[3],item[4]) )
            continue

        if "readme" in item[3].lower() or "changelog" in item[3].lower() or "contributing" in item[3].lower() or "acknowledge" in item[3].lower() or "license" in item[3].lower() or "history" in item[3].lower() or "releasenotes" in item[3].lower():
            third_party.add((item[0], item[1])) # , item[3],item[4]) )
            continue
        
        styling.add((item[0], item[1])) #, item[3],item[4]) )

    return third_party, potential_added_information, styling, all_markdown_in_node_modules




In [ ]:
third_party, potential_added_information, styling, all_markdown_in_node_modules =  get_markdown_stats()

In [ ]:
third_party_24, potential_added_information_24, styling_24, all_markdown_in_node_modules_24 =  get_markdown_stats(duckdb_conn=duckdb_conn_2024)

In [ ]:
all = set()
for item in styling:
    all.add(item)
for item in potential_added_information:
    all.add(item)

In [ ]:
print(all)

In [ ]:
print("third party", get_formated(len(third_party), num_apps))
get_full_stats(third_party)
print("styling", get_formated(len(styling), num_apps))
get_full_stats(styling)
print("node modules", get_formated(len(all_markdown_in_node_modules), num_apps))
get_full_stats(all_markdown_in_node_modules)


In [ ]:
all_md = fetch_all_from_queries(all_markdown)


In [ ]:
get_full_stats(all_md)

In [ ]:
print("2024")
print("third party", get_formated(len(third_party_24), num_apps))
get_full_stats(third_party_24, len_ios=9212, len_android=8702)
print("styling", get_formated(len(styling_24), num_apps))
get_full_stats(styling_24, len_ios=9212, len_android=8702)
print("node modules", get_formated(len(all_markdown_in_node_modules_24), num_apps))
get_full_stats(all_markdown_in_node_modules_24, len_ios=9212, len_android=8702)
all_md = fetch_all_from_queries(all_markdown, duckdb_conn=duckdb_conn_2024)

print(get_full_stats(all_md, len_ios=9212, len_android=8702))

In [ ]:
get_full_stats(potential_added_information.union(styling))
get_full_stats(potential_added_information.union(third_party).union(styling))

In [ ]:
print(potential_added_information)

In [ ]:
# AI - related
get_mlkit_count = "SELECT COUNT(apps.id) FROM apps join files on apps.id = files.app_id WHERE files.suffix = '.tflite' and files.file_path like '%/mlkit_barcode_models/%' ORDER BY platform, file_name;"

In [ ]:
format_data_frame(get_df_for_dot_results(get_hidden_files(), "dot_files"))


In [ ]:
format_data_frame(get_df_for_dot_results(get_hidden_directory_stats(), "dot_directories"))

In [ ]:
format_data_frame(get_df_for_dot_results(get_hidden_files_filtered(), "dot_files_filtered"))


In [ ]:
def get_count_table(data, base_value=0):
    result = {}
    result_android = {}
    result_ios = {}
    for item in data:
        result[item[3]] = result.get(item[3], 0) + 1
        if item[1] == "android":
            result_android[item[3]] = result_android.get(item[3], 0) + 1
            result_ios[item[3]] = result_ios.get(item[3], 0)
        else:
            result_ios[item[3]] = result_ios.get(item[3], 0) + 1
            result_android[item[3]] = result_android.get(item[3], 0)

    print(result_android)
    print(result_ios)


    if base_value > 0:
        for k,v in result.items():
            result[k] = f"{v:,} ({(v / base_value * 100):.2f}%)"
            result_android[k] = f"{result_android[k]:,} ({(result_android[k] / num_android_apps * 100):.2f}%)"
            result_ios[k] = f"{result_ios[k]:,} ({(result_ios[k] / num_ios_apps * 100):.2f}%)"



    df = pd.DataFrame.from_dict(result, orient='index', columns=['occurrences']).sort_values(by='occurrences', ascending=False)
    df['android'] = pd.Series(result_android)
    df['ios'] = pd.Series(result_ios)
    return df


def get_count_table_apps(data, base_value=0):
    result = {}
    result_android = {}
    result_ios = {}
    for item in data:
        current = result.get(item[3], set())
        current.add(item[0])
        result[item[3]] = current
        if item[1] == "android":
            current_android = result_android.get(item[3], set())
            current_ios = result_ios.get(item[3], set())
            current_android.add(item[0])
            result_android[item[3]] = current_android
            result_ios[item[3]] = current_ios
        else:
            current_android = result_android.get(item[3], set())
            current_ios = result_ios.get(item[3], set())
            current_ios.add(item[0])
            result_android[item[3]] = current_android
            result_ios[item[3]] = current_ios


    for k in result.keys():
        result[k] = len(result[k])
        result_android[k] = len(result_android[k])
        result_ios[k] = len(result_ios[k])

    if base_value > 0:
        for k,v in result.items():
            result[k] = f"{v:,} ({(v / base_value * 100):.2f}%)"
            result_android[k] = f"{result_android[k]:,} ({(result_android[k] / num_android_apps * 100):.2f}%)"
            result_ios[k] = f"{result_ios[k]:,} ({(result_ios[k] / num_ios_apps * 100):.2f}%)"



    df = pd.DataFrame.from_dict(result, orient='index', columns=['occurrences']).sort_values(by='occurrences', ascending=False)
    df['android'] = pd.Series(result_android)
    df['ios'] = pd.Series(result_ios)
    return df
    


In [ ]:
hidden_files = get_hidden_files()
hidden_directory_stat = get_hidden_directory_stats()


In [ ]:
get_count_table(hidden_files[0],0 )

In [ ]:

get_count_table_apps(hidden_files[0], 0)

In [ ]:
def map_hidden_directory_to_directory_level(results):
    filtered = {}
    for item in results:
        key = f"{item[0]}-{item[1]}"
        def extract_hidden_dir(file_path):
            splitted = file_path.split("/")
            result = splitted[0]
            for i in range(1, len(splitted)):
                result = result + "/" + splitted[i]
                if splitted[i].startswith(".") and i != len(splitted) -1:
                    return splitted[i]
            return None
        tmp = filtered.get(key, set())
        tmp.add(extract_hidden_dir(item[4]))
        filtered[key] = tmp
    result = []
    for k,v in filtered.items():
        splitted = k.split("-")
        app = splitted[0]
        platform = splitted[1]
        for val in v:
            result.append((app, platform,0, val)) # set to result[3] to reuse count table function

    return result


In [ ]:
query = """
SELECT apps.app_name, COUNT(files.suffix) 
FROM apps 
JOIN files ON apps.id = files.app_id 
WHERE files.suffix = '.swift' 
AND files.app_id IN (SELECT app_id FROM files WHERE file_path LIKE '%.swiftpm%') 
GROUP BY apps.app_name;
"""

result = fetch_all_from_queries(query)
print(result)

In [ ]:
get_count_table_apps(map_hidden_directory_to_directory_level(hidden_directory_stat[0]), num_android_apps + num_ios_apps)

In [ ]:
# 
get_count_table(map_hidden_directory_to_directory_level(hidden_directory_stat[0]), num_android_apps + num_ios_apps)

In [ ]:
src_folder = "SELECT * FROM files join apps on files.app_id = apps.id where file_path like '%/src/%'"

In [ ]:
def get_part(path, part):
    splitted = path.split("/")
    for i in range(1, len(splitted)):
        if part in splitted[i]:
            return splitted[i]
    return None

def get_ml_model_name(item):
    if "model.espresso.weights" in item[2]:
        if ".framework" in item[3]:
            return get_part(item[3], ".framework")
        elif ".mlmodelc" in item[3]:
            return get_part(item[3], ".mlmodelc")
        elif ".bundle" in item[3]:
            return get_part(item[3], ".bundle")
        else:
            return item[2]
    return None
    
        
            

def get_all_ai_files():
    query = "SELECT apps.app_name, apps.platform, files.file_name, files.file_path FROM files JOIN apps ON files.app_id = apps.id WHERE apps.app_name = '' " # pps.app_name = '' to make query easier -> False or ....
    for suffix in suffix_classes.machine_learning:
        query += f"OR files.suffix = '{suffix}' "
    
    return fetch_all_from_queries(query)


    

In [ ]:
def cluster_ai_files(ai_raw_data):
    result = {}
    for item in ai_raw_data:
        model_name = get_ml_model_name(item)
        if model_name is None:
            model_name = item[2]

        tmp = result.get(model_name, set())
        tmp.add((item[0], item[1]))
        result[model_name] = tmp
    return result


def filter_cluster_for_matching(result):
    filtered = {}
    for k,v in result.items():
        tmp = set()
        ignore = []
        for value in v:
            if value[1] == "ios":
                match =  matching_ids.get(value[0], None)
                if match and (match, "android") in v:
                    tmp.add((value[0], "both"))
                    ignore.append(value)
                    ignore.append((match, "android"))
        
        for value in v:
            if value not in ignore:
                tmp.add(value)
        filtered[k] = tmp

                
    return filtered

def filter_cluster_size(result, size = 3):
    filtered = {}
    for k,v in result.items():
        if len(v) < size:
            filtered[k] = v
    return filtered

def invert_map(ai_map):
    result = {}
    for k,v in ai_map.items():
        for value in v:
            key = value[0] + "-" + value[1]
            tmp = result.get(key, set())
            tmp.add(k)
            result[key] = tmp
    return result

In [ ]:
ai_files = get_all_ai_files()



In [ ]:
filtered_matching_ai = filter_cluster_for_matching(cluster_ai_files(ai_files))

In [ ]:
len(invert_map(filter_cluster_size(filtered_matching_ai, size=2)))

In [ ]:
#import networkx as nx
#import matplotlib.pyplot as plt
#G = nx.Graph()

# Add nodes and edges
#for name, apps in filter_cluster_size(filtered_matching_ai).items():
#    G.add_node(name, type='file')  # Add 'name' as a node
#    for app in apps:
#        G.add_node(app, type='app')  # Add 'app' as a node
#        G.add_edge(name, app)  # Add an edge between name and app

# Styling: Differentiate node types
#node_colors = ['blue' if G.nodes[node]['type'] == 'file' else 'green' for node in G.nodes]

# Draw the graph
#plt.figure(figsize=(10, 8))
#nx.draw(G, with_labels=True, node_color=node_colors, font_size=10, font_weight='bold')
#plt.show()

In [ ]:
cluster_ai_files(ai_files)
sorted_ai_files = sorted(cluster_ai_files(ai_files).items(), key=lambda item: len(item[1]), reverse=True)
for key, value in sorted_ai_files:
    print(key, len(value))

In [ ]:
def export(results, name = None, outputName = None, output_prefix = output_dir):
    output = {}
    output_path = os.path.join(output_prefix, outputName)
    if name is None:
        for k,v in results.items():
            for value in v:
                output[f"{value[0]}_{value[1]}"] = k
    else:
        for result in results:
            output[f"{result[0]}_{result[1]}"] = name
    if outputName is not None:
        with open(outputName, "w") as f:
            json.dump(output, f)
    return output


In [ ]:
export(hidden_files[0],  name="hidden_files", outputName="hidden_files_2023.json")
export(hidden_directory_stat[0], name = "hidden_directory_stat", outputName="hidden_directory_stat_2023.json")

In [ ]:
export(get_apps_with_code(code_suffix,duckdb_conn=duckdb_conn_2023), outputName = "code_2023.json")
export(script_stats, outputName = "scripts_2023.json")
export(podfile, name="podfile", outputName="podfile_2023.json")



In [ ]:
export(get_apps_with_code(code_suffix,duckdb_conn=duckdb_conn_2024), outputName = "code_2024.json")


In [ ]:
script_stats_24 = get_apps_with_scripts(script_suffix, duckdb_conn=duckdb_conn_2024)
podfile_24 = fetch_all_from_queries("SELECT DISTINCT app_name, platform FROM files join apps on files.app_id = apps.id where files.file_name like 'Podfile%';",duckdb_conn=duckdb_conn_2024)

In [ ]:
hidden_files_24 = get_hidden_files(duckdb_conn=duckdb_conn_2024)
hidden_directory_stat_24 = get_hidden_directory_stats(duckdb_conn=duckdb_conn_2024)


In [ ]:
export(hidden_files_24[0],  name="hidden_files", outputName="hidden_files_2024.json")
export(hidden_directory_stat_24[0], name = "hidden_directory_stat", outputName="hidden_directory_stat_2024.json")

In [ ]:
export(code_stats_24, outputName = "code_2024.json")
export(script_stats_24, outputName = "scripts_2024.json")
export(podfile_24, name="podfile", outputName="podfile_2024.json")

In [ ]:
google_service_query = "Select DISTINCT apps.app_name, apps.platform from apps join files on apps.id = files.app_id where files.file_name like 'google-services%.json' or files.file_name like 'GoogleService-Info%.plist';"
google_service_2023 = fetch_all_from_queries(google_service_query)
google_service_2024 = fetch_all_from_queries(google_service_query, duckdb_conn=duckdb_conn_2024)

In [ ]:
env_query = "Select DISTINCT apps.app_name, apps.platform from apps join files on apps.id = files.app_id where (files.file_name like '%.env' or files.file_name like '.env%') and files.mime_type = 'text/plain';"
env_2023 = fetch_all_from_queries(env_query)
env_2024 = fetch_all_from_queries(env_query, duckdb_conn=duckdb_conn_2024)

In [ ]:
xcodeconf_query = "Select DISTINCT apps.app_name, apps.platform from apps join files on apps.id = files.app_id where (files.file_name like '%.xcconfig');"
xcodeconf_2023 = fetch_all_from_queries(xcodeconf_query)
xcodeconf_2024 = fetch_all_from_queries(xcodeconf_query, duckdb_conn=duckdb_conn_2024)

In [ ]:
len(xcodeconf_2023)

In [ ]:
len(xcodeconf_2024)